In [ ]:
from pynq import PL
PL.reset() #important fixes caching issues which have popped up.
from pynq import Overlay  #import the overlay module
ol = Overlay('./design_1_wrapper.bit')  #locate/point to the bit file

In [ ]:
import math
fi = ol.fir_interface_0  #find the AXI MMIO module which we can talk to (name of IP)
coeffs = []
emboss = [0,0,0,0,0,0,0,0,0,0,0,0,0,-4,4] #simple emboss kernel
smooth=[-2,-3,-4,0,9,21,32,36,32,21,9,0,-4,-3,-2]
jumpy=[-3,14,-20,6,16,-5,-41,68,-41,-5,16,6,-20,14,-3]

coeffs = emboss

#most kernels will have > unity gain. We need to undo this with a final shift.
#to calculate, sum the coefficients
#figure out the closest power 2 that sum is, then use to assign the shift amount
coeffs_sum = sum(coeffs)
shift_amt = 0
if coeffs_sum!=0:
    shift_amt = int(math.ceil(math.log(sum(coeffs),2)))
print(coeffs_sum, shift_amt)
#To set values in our MMIO:
 
#write coefficients (registers 0 through 15):
for i in range(15):
    fi.write(i*4,coeffs[i])
fi.write(15*4,7) #turn on all three channels (rgb) via reg 15
fi.write(16*4,shift_amt) #write shift amount to reg 16
fi.write(17*4,1) #turn on offset signal (signals processed as -128 to 127) (register 17)
d = fi.read(15*4) #read back for sanity
print(hex(d))